# trajectory_analyze

particle trajectory analysis

file requirements:
- particle netcdf file generated by parcels after simulation
- the actual ocean data the particles ran on

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
from datetime import timedelta, datetime
import math
import cartopy.crs as ccrs
import cartopy
import xarray as xr
import numpy as np
from parcels import FieldSet, ParticleSet, JITParticle
import matplotlib.pyplot as plt

from utils import conv_to_dataarray, generate_mask, euc_dist, haversine
from parcels_utils import get_file_info, xr_dataset_to_fieldset
from plot_utils import plot_particles, get_carree_axis

In [ ]:
# lOLOLOLOLOLOL
coastline_points = np.array([
    [32.527430, -117.123895],
    [32.534549, -117.123832],
    [32.549035, -117.125920],
    [32.556846, -117.130490],
    [32.560474, -117.131934],
    [32.562632, -117.132144],
    [32.564511, -117.132933],
    [32.575317, -117.132941],
    [32.576679, -117.133227],
    [32.593312, -117.133008],
    [32.616745, -117.136719],
    [32.640275, -117.146353],
    [32.662540, -117.160633],
    [32.679572, -117.180263],
    [32.687357, -117.198503],
    [32.683428, -117.223487]
])

tijuana_mouth = np.array([32.551967, -117.127208])

In [ ]:
# ocean current netcdf data the particles used
d_nc = "current_netcdfs/west_coast_1km_hourly/tijuana_river_lin.nc"
# the data generated by ParticleFile
p_nc = "particledata/particle_tijuana_lin.nc"

In [ ]:
d_info = get_file_info(d_nc, 1)
d_ds = xr.open_dataset(d_nc)
p_ds = xr.open_dataset(p_nc)
fs = xr_dataset_to_fieldset(d_ds)
d_ds.close()
p_ds.close()

In [ ]:
ax = get_carree_axis(d_info["domain"])
# plt.scatter(coastline_points.T[1], coastline_points.T[0])
plt.plot(coastline_points.T[1], coastline_points.T[0])

so this doesn't actually find the closest distance to the coastline, but the closest distance to one of the points defined to be the coastline because it's easier

dists = np.empty((p_ds.dims["traj"], p_ds.dims["obs"]))
for i in range(dists.shape[0]):
    for j in range(dists.shape[1]):
        if np.isnan(p_ds["trajectory"].isel(traj=i, obs=j)):
            dists[i, j:] = np.nan
            break
        lat = p_ds["lat"].isel(traj=i, obs=j)
        lon = p_ds["lon"].isel(traj=i, obs=j)
        p = np.array([lat, lon])
#         point_dists = [euc_dist(p, coast_p) for coast_p in coastline_points]
#         closest = coastline_points[np.where(point_dists == min(point_dists))]
        closest = np.array([tijuana_mouth])
        dists[i, j] = haversine(lat, closest[0, 0], lon, closest[0, 1])  # meters

dists

In [ ]:
d_mask = conv_to_dataarray(generate_mask(d_ds["u"].values), d_ds["u"])

### check whether a particle collided with land

conditions:
- only check the latest position of a particle before it's deleted or stuck on land or something
- particle is on a position where even after fieldset interpolation, vector values are still zero

however, there may be some regions of actual ocean that just don't have data, so it's not foolproof

### edit

got a long ass list of coordinates representing the coastline now, and all the points will essentially be on the "bottom-left" of the coastline, so easy check i guess

In [ ]:
def collided_with_coastline(lat, lon):
    upmost = coastline_points.T[0].max()
    rightmost = coastline_points.T[1].max()
    return lat > upmost or lon > rightmost


def time_to_str(time):
    """
    haha time go chop chop
    
    Args:
        time (np.datetime64)
    """
    return str(time).split(".")[0]

### some immediate problems

- particles are out of bounds even though I use coordinate values on the SAME FIELDSET during simulation
    - ??????????????? how


In [ ]:
collided_land = np.zeros(p_ds["trajectory"].shape, dtype=bool)
lat_ls_coll = []
lon_ls_coll = []
times_sec = []
times_dates = []
part_dists = []
for p_iter in range(len(p_ds["trajectory"])):
    p = p_ds["trajectory"][p_iter]
    # find the first state of the particle where it got stuck on land something
    for step in range(len(p)):
        if np.isnan(p[step]):
            break
        time = p_ds["time"].values[p_iter, step]
        time_secs = p_ds["lifetime"].values[p_iter, step] + p_ds["spawntime"].values[p_iter, step]
        lat = p_ds["lat"].values[p_iter, step]
        lon = p_ds["lon"].values[p_iter, step]
        times_sec.append(time_secs)
        times_dates.append(time)
#         part_dists.append(dists[p_iter, step])
        # first, check if the vector speed at that coordinate is 0 after interpolation
        # second, check if the particle is not in an invalid coordinate (a coordinate supposed to have data)
        total_vel = abs(fs.U[time_secs, 0, lat, lon]) + abs(fs.V[time_secs, 0, lat, lon])
        if collided_with_coastline(lat, lon):
#         if total_vel <= sys.float_info.epsilon and not d_mask.sel(time=time, lat=lat, lon=lon, method="nearest"):
            lat_ls_coll.append(lat)
            lon_ls_coll.append(lon)
            print(f"particle {p_iter} beached at time {time_to_str(time)}")
            print(f"    spawn time: {time_to_str(p_ds['time'].values[p_iter, 0])}")
            print(f"    spawn coord: ({p_ds['lat'].values[p_iter, 0]}, {p_ds['lon'].values[p_iter, 0]})")
            print(f"    lifetime: {p_ds['lifetime'].values[p_iter, step]}")
            break

30, 31, 32, 33, 34, 37, 38, 39

plt.hist2d(times_sec, part_dists)
plt.show()

In [ ]:
lats_all = []
lons_all = []
age_all = []
final_time = d_ds["time"].max().values
for p_iter in range(len(p_ds["trajectory"])):
    p = p_ds["trajectory"][p_iter]
    for step in reversed(range(len(p))):
        if not np.isnan(p[step]) and p_ds["time"].values[p_iter, step] == final_time:
            lats_all.append(p_ds["lat"].values[p_iter, step])
            lons_all.append(p_ds["lon"].values[p_iter, step])
            age_all.append(p_ds["lifetime"].values[p_iter, step])
            break

In [ ]:
plot_particles(fs, lat_ls_coll, lon_ls_coll)

In [ ]:
d_info["domain"]

In [ ]:
def show_particles_age(lats, lons, ages, domain):
    ext = [domain["W"], domain["E"], domain["S"], domain["N"]]
    
    ax = plt.axes(projection=ccrs.PlateCarree())
    ax.set_extent(ext)
    ax.add_feature(cartopy.feature.COASTLINE)

    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True)
    gl.top_labels, gl.right_labels = (False, False)
    gl.xformatter = cartopy.mpl.gridliner.LONGITUDE_FORMATTER
    gl.yformatter = cartopy.mpl.gridliner.LATITUDE_FORMATTER

    plt.scatter(lons, lats, c=ages, edgecolors='k', vmin=0)
    plt.colorbar()
    plt.title(f"Particle ages")
    plt.show()

In [ ]:
seconds_to_days = 1 / 86400
age_days = np.array(age_all) * seconds_to_days
# print(age_days)
d = d_info["domain"]

show_particles_age(lats_all, lons_all, age_days, d)